In [ ]:
from odt_parse import OdtData
from odt_diff import find_style_by_name

In [ ]:
import csv, glob, zipfile

In [ ]:
HOME_FOLDER = '../'
SUBMISSION_FOLDER = HOME_FOLDER + 'PS1002-2-20172018-Document libro modificado.odt, grups de dijous-3219490'
user_folder = glob.glob(SUBMISSION_FOLDER + '/*')

In [ ]:
ref_name = 'libro_modificado.odt'
par_prop = ['backgroundcolor', 'textalign', 'marginleft', 'marginright', 'margintop', 'marginbottom']
text_prop = ['fontsize', 'fontstyle']
ref = OdtData(ref_name, par_prop, text_prop)

In [ ]:
errors = []
sp_dict = {'Heading':'Título', 'backgroundcolor':'color de fondo', 'textalign':'alineación', 
           'Quotations':'Cita', 'fontsize':'tamaño de letra', 'fontstyle':'efecto tipográfico', 
           'marginleft':'sangría izquierda', 'marginright':'sangría derecha', 
           'margintop':'espacio superior', 'marginbottom':'espacio inferior'}
def compare_style_attr(ref, doc, family, style_name, attr_list):
    stref = find_style_by_name(ref.style[family], style_name)
    stdoc = find_style_by_name(doc.style[family], style_name)
    for attr in attr_list:
        try:
            val_ref = stref[attr]
            try:
                val_doc = stdoc[attr]
                if val_ref != val_doc:
                    f.write('Estilo %s tiene %s \n  %s en lugar de %s.\n\n' % (sp_dict[style_name], sp_dict[attr],
                                                                                 val_doc, val_ref))
            except KeyError:
                    f.write('Estilo %s no tiene %s definido.\n\n' % (sp_dict[style_name], sp_dict[attr]))
            except TypeError:
                    f.write('Estilo %s no está definido.\n\n' % (sp_dict[style_name]))
        except KeyError:
            err = style_name + "_" + attr
            if not err in errors:
                errors.append(err)
                print('Estilo %s no tiene %s definido en el fichero de referencia.' % (style_name, attr))

In [ ]:
def compare_style_attrs():
    compare_style_attr(ref, doc, 'paragraph', 'Heading', 
                       ['backgroundcolor', 'textalign'])
    compare_style_attr(ref, doc, 'paragraph', 'Quotations', 
                       ['fontsize', 'fontstyle', 'textalign', 'marginleft', 'marginright', 'margintop', 'marginbottom'])

In [ ]:
with open(SUBMISSION_FOLDER + '.txt', 'w', encoding='utf-8') as f:
    counter = 0
    for ufo in user_folder:
        user_files = glob.glob(ufo + '/*')
        for ufi in user_files:
            tokens = ufi.split('/')
            filename = tokens[-1]
            user_data = tokens[-2]
            user_name, user_id, _, submission_type, _ = user_data.split('_')
            fns = filename.split('.')
            if len(fns) > 1:
                extension = fns[-1]
            else:
                extension = ''
            row = [user_name, user_id, submission_type, filename, extension]
            f.write(user_name + '\n' + '-'*len(user_name) + '\n')
            if filename != ref_name:
                f.write('\nNombre de fichero: %s\n\n' % filename)
            if extension == 'odt':
                doc = OdtData( ufi, par_prop, text_prop )
                if doc.err:
                    f.write('Error de lectura de fichero\n')                   
                else:
                    compare_style_attrs()      
            else:
                f.write('Extensión de fichero incorrecta\n')
            f.write('\n' + '#'*40 + '\n\n')
            counter += 1
    print('%d ficheros procesados.' % counter)